In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install datasets

In [ ]:
import evaluate

In [ ]:

from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
data=pd.read_csv('data.csv')

In [ ]:
data=data.rename(columns={"word_labels": "label"})
data

,Unnamed: 0,sentence,label
0,0,Account Number : 654321098,"B-key,I-key,I-key,B-val"
1,1,Amount : $1800.80,"B-key,I-key,B-val"
2,2,"Date of Birth : June 12, 1975","B-key,I-key,I-key,I-key,B-val,I-val,I-val"
3,3,Social Security Number : 654-32-1098,"B-key,I-key,I-key,I-key,B-val"
4,4,Membership ID : M23456,"B-key,I-key,I-key,B-val"
...,...,...,...
650,650,Investment Acct. No. 789012345,"B-key,I-key,I-key,B-val"
651,651,Savings Acct. Holder Sarah Martinez,"B-key,I-key,I-key,B-val,I-val"
652,652,Savings Acct. Type High-Yield Savings,"B-key,I-key,I-key,B-val,I-val"
653,653,Savings Acct. No. 456789012,"B-key,I-key,I-key,B-val"


In [ ]:
data['sentence'][2].split()

['Date', 'of', 'Birth', ':', 'June', '12,', '1975']

In [ ]:
label2id={'O': 0, 'B-key': 1, 'I-key': 2, 'B-val': 3, 'I-val': 4}

In [ ]:
def convert_labels(label_string, label_mapping=label2id):
    # Split the string into individual labels
    labels = label_string.split(',')

    # Map each label to its numeric value
    numeric_labels = [label_mapping[label] for label in labels]

    return numeric_labels

# Example usage:
label_string = 'B-key,I-key,I-key,B-val,I-val'
label_mapping = {'B-key': 1, 'I-key': 2, 'B-val': 3, 'I-val': 4}

result = convert_labels(label_string)
print(result)

def split1(sentence):
    return sentence.split()

[1, 2, 2, 3, 4]


In [ ]:
data['label']=data['label'].apply(convert_labels)
data['sentence']=data['sentence'].apply(split1)

In [ ]:
#pip install-r requirements.txt
from datasets import DatasetDict


In [ ]:


# Convert to the desired format
converted_data = {
    "id": [str(i) for i in range(len(data["sentence"]))],
    "tokens": data["sentence"],
    "ner_tags": data["label"]
}

# Create a Dataset using the converted data
ner_dataset = Dataset.from_dict(converted_data)

# Split the dataset into training, validation, and testing sets
train_size = 0.8
validation_size = 0.1
test_size = 0.1

train_dataset, test_dataset = train_test_split(ner_dataset, test_size=0.3, random_state=42)
validation_dataset= test_dataset

# Create a DatasetDict
wnut = DatasetDict({
    "train": Dataset.from_dict(train_dataset),
    "validation": Dataset.from_dict(validation_dataset),
    "test": Dataset.from_dict(test_dataset)
})

# Print the sizes of the resulting subsets
print("Training set size:", len(train_dataset['id']))
print("Validation set size:", len(validation_dataset['id']))
print("Testing set size:", len(test_dataset['id']))


Training set size: 458
Validation set size: 197
Testing set size: 197


Each number in `ner_tags` represents an entity. Convert the numbers to their label names to find out what the entities are:

In [ ]:
label_list = ['O',
 'B-key',
 'I-key','B-val','I-val']

## Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the `tokens` field:

In [ ]:
from transformers import AutoTokenizer
model_name='roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name,add_prefix_space=True)

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
wnut["train"][0]

{'id': '417',
 'tokens': ['Return', 'Date', 'November', '5,', '2023'],
 'ner_tags': [1, 2, 3, 4, 4]}

In [ ]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>', 'ĠReturn', 'ĠDate', 'ĠNovember', 'Ġ5', ',', 'Ġ20', '23', '</s>']

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],padding=True, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/458 [00:00<?, ? examples/s]

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 406.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a8c2ee7fce7d888d9f3e2fad789a6b35f511c7dd24891e381e48ac3822f26d8c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['B-key', 'I-key', 'B-val', 'I-val', 'I-val']

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [ ]:

id2label = {
    0: 'O',
    1: 'B-key',
    2: 'I-key',
    3: 'B-val',
    4: 'I-val',
}

label2id = {v: k for k, v in id2label.items()}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

data_collator=data_collator,
    compute_metrics=compute_metrics,

In [ ]:
training_args = TrainingArguments(
    output_dir="roberta_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.061549,0.954887,0.967005,0.960908,0.980884
2,No log,0.034652,0.984772,0.984772,0.984772,0.991637


TrainOutput(global_step=58, training_loss=0.28105291826971646, metrics={'train_runtime': 946.6343, 'train_samples_per_second': 0.968, 'train_steps_per_second': 0.061, 'total_flos': 29907571559472.0, 'train_loss': 0.28105291826971646, 'epoch': 2.0})

<Tip>

For a more in-depth example of how to finetune a model for token classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification-tf.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [ ]:
from transformers import pipeline

pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")


In [ ]:
pipe('account no 66868 -67 .877')

[{'entity_group': 'key',
  'score': 0.9996495,
  'word': ' account no',
  'start': 0,
  'end': 10},
 {'entity_group': 'val',
  'score': 0.94745386,
  'word': ' 66868 -67 .877',
  'start': 11,
  'end': 25}]

# Given data
data = {
    "sentence": [
        ['amount', 'paid', ':', '$', '2', ',', '500', '.', '00'],
        ['amount', 'given', ':', '$', '7', ',', '300', '.', '00'],
        ['amount', 'paid', ':', '$', '2', ',', '500', '.', '00'],
        ['amount', 'paid', ':', '$', '9', ',', '500', '.', '00'],
        ['amount', 'paid', ':', '$', '2', ',', '500', '.', '00'],
        ['amount', 'given', ':', '$', '7', ',', '300', '.', '00'],
        ['amount', 'paid', ':', '$', '2', ',', '500', '.', '00'],
        ['amount', 'paid', ':', '$', '9', ',', '500', '.', '00'],
        ['amount', 'paid', ':', '$', '2', ',', '500', '.', '00'],
        ['amount', 'given', ':', '$', '7', ',', '300', '.', '00'],
        ['amount', 'paid', ':', '$', '2', ',', '500', '.', '00'],
        ['amount', 'paid', ':', '$', '9', ',', '500', '.', '00']
    ],
    "label": [
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4],
        [1, 2, 2, 3, 4, 4, 4, 4, 4]
    ]
}